In [584]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [585]:
actual = pd.read_pickle('actual.pkl')
predictions = pd.read_pickle('predictions.pkl')

file_paths = actual['file_path']
arousal = actual['arousal']
valence = actual['valence']

In [586]:
model = 'Video_M_1'
f = open('ready_models_video/results/video_predict.pkl', 'rb')
data = pickle.load(f)
f.close()

In [567]:
pred = pd.DataFrame(data, columns=['valence_{}'.format(model), 'arousal_{}'.format(model)])
pred = pd.concat([file_paths, pred], axis = 1)

In [568]:
# predictions.merge(pred, on = 'file_path') #add inplace

In [569]:
def get_clip(path):
    file = path.split('\\')[:-1]
    return '\\'.join(file)

In [570]:
pred['file_path'] = pred['file_path'].apply(get_clip)

clip_pred_valence = pred.groupby('file_path', sort = False)['valence_{}'.format(model)].mean()
valence_df = pd.DataFrame(clip_pred_valence.values, columns = ['valence'])

files = pd.DataFrame(list(pred.groupby('file_path', sort = False).groups.keys()), columns = ['file_path'])

clip_pred_arousal = pred.groupby('file_path', sort = False)['arousal_{}'.format(model)].mean()
arousal_df = pd.DataFrame(clip_pred_arousal.values, columns = ['arousal'])

In [571]:
pred_final = pd.concat([files, valence_df, arousal_df], axis = 1)

In [572]:
actual['file_path'] = actual['file_path'].apply(get_clip)

clip_actual_valence = actual.groupby('file_path', sort = False)['valence'].mean()
valence_df = pd.DataFrame(clip_actual_valence.values, columns = ['valence'])

files = pd.DataFrame(list(actual.groupby('file_path', sort = False).groups.keys()), columns = ['file_path'])

clip_actual_arousal = actual.groupby('file_path', sort = False)['arousal'].mean()
arousal_df = pd.DataFrame(clip_actual_arousal.values, columns = ['arousal'])

In [573]:
actual_final = pd.concat([files, valence_df, arousal_df], axis = 1)

In [574]:
# actual_final.to_pickle('actual_mean.pkl')
# pred_final.to_pickle('pred_mean.pkl')

In [575]:
def error(actual, predict, class_num):
    pred = predict[predict.columns[class_num]].values
    act = actual[actual.columns[class_num]].values
    all = len(act)
    pos = 0
    for i in range(len(act)):
        a = act[i]
        p = pred[i]
        if abs(a - p) < 0.1:
            pos += 1
    return pos/all

In [576]:
# fin = actual.merge(pred, on = 'file_path')

# actual = fin[['valence', 'arousal']]

# pred = fin[['valence_Video_M_1', 'arousal_Video_M_1']]

In [553]:
# print('valence error by frames = {}'.format(error(actual, pred, 1)))
# print('arousal error by frames = {}'.format(error(actual, pred, 2)))

In [556]:
fin = actual_final.merge(pred_final, on = 'file_path')

actual = fin[['valence_x', 'arousal_x']]

pred = fin[['valence_x', 'arousal_x']]

In [562]:
print('valence error by frames = {}'.format(error(actual_final, pred_final, 1)))
print('arousal error by frames = {}'.format(error(actual_final, pred_final, 2)))

valence error by frames = 0.26991150442477874
arousal error by frames = 0.2168141592920354


In [583]:
pred_final.to_pickle('mean_pred_video.pkl')

In [497]:
# from scipy.stats import pearsonr
# import torch
# import torch.nn.functional as F

In [498]:
# pred_V = pred[pred.columns[1]].values.reshape(len(pred.values[:,1]),1)
# pred_A = pred[pred.columns[2]].values.reshape(len(pred.values[:,2]),1)
# predict = np.hstack([pred_V, pred_A])

In [499]:
# actual_V = actual[actual.columns[1]].values.reshape(len(actual.values[:,1]),1)
# actual_A = actual[actual.columns[2]].values.reshape(len(actual.values[:,2]),1)
# actual = np.hstack([actual_V, actual_A])

In [500]:
# result = []
# for k, name in enumerate(['Valence', 'Arousal']):
#     target = torch.from_numpy(actual[:,k]).double()
#     pred = torch.from_numpy(predict[:,k]).double()
#     test_acc = torch.nonzero(F.relu(-(target - pred).abs_() + 0.1)).size(0)
#     test_acc *= 100 / actual.shape[0]

#     result.append(test_acc)
#     print('   accuracy per frames: %0.3f%%' % test_acc)

   accuracy per frames: 27.707%
   accuracy per frames: 20.877%
